In [ ]:
from ipykernel.kernelspec import install
import pip
import ipykernel
#Needed to read xlsb files from model - if not installed
#pip install pyxlsb


In [12]:
import pandas as pd #For analysis
import numpy as np
import pyodbc #To connect to the PT database
import pyxlsb #To open an .xlsb file
from IPython.display import display, HTML #To display a better output for summary tables at the bottom
from sqlalchemy import create_engine
import urllib

In [16]:
# Create DB connection to read directly from PT database in MS SQL SERVER

cnxn_str = ("Driver={ODBC Driver 17 for SQL Server};"
            "Server=172.17.66.113,1433;"
            "Database=MDAnalyst;"
            "Trusted_Connection=yes;"
)
params = urllib.parse.quote_plus(cnxn_str)
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")
cnxn = engine.connect()

#cnxn = pyodbc.connect(cnxn_str)

In [17]:
#Test Query
sql_query="""
SELECT distinct [MRG Market]
FROM PriceTrack_Complete_NT_View Where [MRG Market] like '%Large%'
"""

mrg_market_list = pd.read_sql(sql_query,cnxn)
print(mrg_market_list)

          MRG Market
0  Large Joint Recon


In [19]:
#Declaring variables - change these based on your needs
#Module you want to map from PT
module = "Large Joint Recon"
#Starting date for PT data
date = "01/01/2018"
#Revenue threshold above which you will map SKUs
rev_thres = 1000000 
#Latest year with complete data in PT data
curr_year = 2024
#Path to Data file with PT mapping
map_path = "C:/Users/U6065175/Clarivate Analytics/DRG Sharing AMERS - Ortho/Rahul Paul's Inbox/Large Joint Recon/US Large Joint Reconstructive Implants/2026/Model/"
#Name of data file with PT mapping
data_filenm = "47042594_Data.xlsb" 
#Path of script output files 
out_path = "C:/Users/U6065175/Clarivate Analytics/DRG Sharing AMERS - Ortho/Rahul Paul's Inbox/Large Joint Recon/US Large Joint Reconstructive Implants/2026/Internal Resources/PT/"



In [20]:
#The query for the pt Module - modify this for your module
sql_query = """SELECT [ID] as deviceID
      ,[MRG Market]
      ,[Product Type]
      ,[Product Subtype 1]
      ,[Product Subtype 2]
      ,[Product Subtype 3]
      ,[Product Subtype 4]
      ,[Product Subtype 5]
      ,[Product Subtype 6]
      ,[Product Subtype 7]
      ,[Product Subtype 8]
      ,[Diameter]
      ,[Length]
      ,[Width]
      ,[Catheter]
      ,[Year]
      ,[Quarter]
      ,[TransDate]
      ,[US Census Region]
      ,[Manufacturer]
      ,[Family]
      ,[Model]
      ,[SKU]
      ,[Units]
      ,[Revenue]
  FROM PriceTrack_Complete_NT_View
  WHERE [MRG Market] LIKE '%""" + module +"""%'
  AND TransDate > '""" + date + """'"""

#Check the query string - modify if needed
print(sql_query)



SELECT [ID] as deviceID
      ,[MRG Market]
      ,[Product Type]
      ,[Product Subtype 1]
      ,[Product Subtype 2]
      ,[Product Subtype 3]
      ,[Product Subtype 4]
      ,[Product Subtype 5]
      ,[Product Subtype 6]
      ,[Product Subtype 7]
      ,[Product Subtype 8]
      ,[Diameter]
      ,[Length]
      ,[Width]
      ,[Catheter]
      ,[Year]
      ,[Quarter]
      ,[TransDate]
      ,[US Census Region]
      ,[Manufacturer]
      ,[Family]
      ,[Model]
      ,[SKU]
      ,[Units]
      ,[Revenue]
  FROM PriceTrack_Complete_NT_View
  WHERE [MRG Market] LIKE '%Large Joint Recon%'
  AND TransDate > '01/01/2018'


In [23]:
#Run the query and display a summary of the data retrieved
pt_data = pd.read_sql(sql_query,cnxn)
#Standardize on product type and manufacturer
pt_data['Manufacturer'] = pt_data['Manufacturer'].str.title()
pt_data['Product Type'] = pt_data['Product Type'].str.title()
print(pt_data.describe())
print(pt_data.info())

OperationalError: (pyodbc.OperationalError) ('08S01', '[08S01] [Microsoft][ODBC Driver 17 for SQL Server]TCP Provider: An existing connection was forcibly closed by the remote host.\r\n (10054) (SQLGetData); [08S01] [Microsoft][ODBC Driver 17 for SQL Server]Communication link failure (10054)')
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [22]:
#Aggregate data annually to filter out SKUs with annual revenue less than the threshold specified
pt_annual =  pt_data.groupby(['deviceID', 
                           'Year', 
                           'MRG Market', 
                           'Product Type', 
                           'Product Subtype 1',
                           'Product Subtype 2', 
                           'Product Subtype 3',
                           'Product Subtype 4',
                           'Diameter',
                           'Length',
                           'Width',
                           'Manufacturer',
                           'Family',
                           'Model',
                           'SKU'], dropna = False).agg(Units = ('Units','sum'), Revenue = ('Revenue', 'sum')).reset_index()
#Standardize on product type and manufacturer
pt_annual['Manufacturer'] = pt_annual['Manufacturer'].str.title()
pt_annual['Product Type'] = pt_annual['Product Type'].str.title()
display(pt_annual.head(10))
#Filter out SKUs with annual revenue lower than specified threshold
curr_sku_list = pt_annual.loc[(pt_annual["Revenue"] > rev_thres) & (pt_annual["Year"] == curr_year), ]
print(curr_sku_list.describe())
print(curr_sku_list.head())




NameError: name 'pt_data' is not defined

In [10]:
#Retrieve current map from PT model file
#Adjust the header and skiprows to ensure it reads the table from the correct row 
pt_map = pd.read_excel(map_path + data_filenm, sheet_name = 'PTMap', engine = 'pyxlsb', header=1, skiprows=range(1,1))
pt_map = pt_map.replace('|||||||||', np.NaN)
print(pt_map.info())
print(pt_map.describe())
taxonomy = pt_map[["Taxonomy",
                   "T1_Type",
                   "T1",
                   "T2_Type",
                   "T2",
                   "T3_Type",
                   "T3",
                   "T4_Type",
                   "T4",
                   "T5_Type",
                   "T5"
                  ]].drop_duplicates()

print(taxonomy)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30262 entries, 0 to 30261
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   deviceID           30262 non-null  int64  
 1   Status             0 non-null      float64
 2   Comments           0 non-null      float64
 3   Taxonomy           22660 non-null  object 
 4   T1_Type            22660 non-null  object 
 5   T1                 22660 non-null  object 
 6   T2_Type            22660 non-null  object 
 7   T2                 22660 non-null  object 
 8   T3_Type            22660 non-null  object 
 9   T3                 22660 non-null  object 
 10  T4_Type            22658 non-null  object 
 11  T4                 22658 non-null  object 
 12  T5_Type            11225 non-null  object 
 13  T5                 11225 non-null  object 
 14  SKU                30262 non-null  object 
 15  Manufacturer       30262 non-null  object 
 16  Family             302

In [11]:
#Combines the pt data with the map file to refresh the map and create a mapped data file
result = pd.merge(curr_sku_list[['deviceID',
                                 'SKU',
                                 'Manufacturer',
                                 'Family',
                                 'Model',
                                 'MRG Market', 
                                 'Product Type', 
                                 'Product Subtype 1',
                                 'Product Subtype 2', 
                                 'Product Subtype 3',
                                 'Product Subtype 4'
                               ]], pt_map[['deviceID',
                                           "Status",
                                           "Comments",
                                           "Taxonomy",
                                           "T1_Type",
                                           "T1",
                                           "T2_Type",
                                           "T2",
                                           "T3_Type",
                                           "T3",
                                           "T4_Type",
                                           "T4",
                                           "T5_Type",
                                           "T5"
                                          ]], how="left", on=["deviceID"])
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1440 entries, 0 to 1439
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   deviceID           1440 non-null   int64  
 1   SKU                1440 non-null   object 
 2   Manufacturer       1440 non-null   object 
 3   Family             1440 non-null   object 
 4   Model              1440 non-null   object 
 5   MRG Market         1440 non-null   object 
 6   Product Type       1440 non-null   object 
 7   Product Subtype 1  1424 non-null   object 
 8   Product Subtype 2  1202 non-null   object 
 9   Product Subtype 3  1310 non-null   object 
 10  Product Subtype 4  324 non-null    object 
 11  Status             0 non-null      float64
 12  Comments           0 non-null      float64
 13  Taxonomy           1216 non-null   object 
 14  T1_Type            1216 non-null   object 
 15  T1                 1216 non-null   object 
 16  T2_Type            1216 

In [12]:
#Creating maps and reordering columns to match existing order
pt_unmapped = result[result["Taxonomy"].isnull()]
pt_mapped = result[result["Taxonomy"].notnull()]
pt_unmapped = pt_unmapped.iloc[:,[0,11,12,13,14,15,16,17,18,19,20,21,22,23,1,2,3,4,5,6,7,8,9,10]]
pt_mapped = pt_mapped.iloc[:,[0,11,12,13,14,15,16,17,18,19,20,21,22,23,1,2,3,4,5,6,7,8,9,10]]
print(pt_unmapped.info())
print(pt_mapped.info())

<class 'pandas.core.frame.DataFrame'>
Index: 224 entries, 1 to 1275
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   deviceID           224 non-null    int64  
 1   Status             0 non-null      float64
 2   Comments           0 non-null      float64
 3   Taxonomy           0 non-null      object 
 4   T1_Type            0 non-null      object 
 5   T1                 0 non-null      object 
 6   T2_Type            0 non-null      object 
 7   T2                 0 non-null      object 
 8   T3_Type            0 non-null      object 
 9   T3                 0 non-null      object 
 10  T4_Type            0 non-null      object 
 11  T4                 0 non-null      object 
 12  T5_Type            0 non-null      object 
 13  T5                 0 non-null      object 
 14  SKU                224 non-null    object 
 15  Manufacturer       224 non-null    object 
 16  Family             224 non-nul

In [13]:
#Preview maps - optional
print(pt_unmapped.head())
print(pt_mapped.head())

    deviceID  Status  Comments Taxonomy T1_Type   T1 T2_Type   T2 T3_Type  \
1     155207     NaN       NaN      NaN     NaN  NaN     NaN  NaN     NaN   
5     155762     NaN       NaN      NaN     NaN  NaN     NaN  NaN     NaN   
10    156297     NaN       NaN      NaN     NaN  NaN     NaN  NaN     NaN   
17    156691     NaN       NaN      NaN     NaN  NaN     NaN  NaN     NaN   
22    157941     NaN       NaN      NaN     NaN  NaN     NaN  NaN     NaN   

     T3  ...             SKU      Manufacturer             Family  \
1   NaN  ...        71935387  Smith And Nephew             Legion   
5   NaN  ...     CAP-MED-HIP           Medacta  Medacta Capitated   
10  NaN  ...        71424225  Smith And Nephew             Legion   
17  NaN  ...  42-5221-007-10     Zimmer Biomet      Zimmer Biomet   
22  NaN  ...      5560-S-215           Stryker          Triathlon   

                Model         MRG Market Product Type       Product Subtype 1  \
1              Legion  Large Joint Recon 

In [14]:
#Use trimmed version of previous mapping file to create a data file
map_trim =pt_map.drop(['SKU',
                       'Family',
                       'Model',
                       'MRG Market',
                       'Product Type',
                       'Product Subtype 1',
                       'Product Subtype 2',
                       'Product Subtype 3',
                       'Product Subtype 4',
                       'T1_Type',
                       'T1',
                       'T2_Type',
                       'T2',
                       'T3_Type',
                       'T3',
                       'T4_Type',
                       'T4',
                       'T5_Type',
                       'T5',
                       'Status',
                       'Comments'
                      ], axis = 1)
map_trim.rename(columns={"Manufacturer": "Company"}, inplace=True)

print(map_trim.info())
print(map_trim.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30262 entries, 0 to 30261
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   deviceID  30262 non-null  int64 
 1   Taxonomy  22660 non-null  object
 2   Company   30262 non-null  object
dtypes: int64(1), object(2)
memory usage: 709.4+ KB
None
            deviceID
count   30262.000000
mean   266332.236567
std     81829.977211
min       637.000000
25%    202399.000000
50%    251117.500000
75%    303678.500000
max    521801.000000


In [15]:
# Group data quarterly
pt_quarter =  pt_data.groupby(['deviceID',
                               'Year',
                               'Quarter',
                               'MRG Market', 
                               'Product Type', 
                               'Product Subtype 1',
                               'Product Subtype 2', 
                               'Product Subtype 3',
                               'Product Subtype 4',
                               'Manufacturer',
                               'Family',
                               'Model',
                               'SKU'], dropna = False).agg(Units = ('Units','sum'), Revenue = ('Revenue', 'sum')).reset_index()
#Standardize on product type and manufacturer
pt_quarter['Manufacturer'] = pt_quarter['Manufacturer'].str.title()
pt_quarter['Product Type'] = pt_quarter['Product Type'].str.title()
print(pt_quarter.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 551121 entries, 0 to 551120
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   deviceID           551121 non-null  int64  
 1   Year               551121 non-null  int64  
 2   Quarter            551121 non-null  int64  
 3   MRG Market         551121 non-null  object 
 4   Product Type       551121 non-null  object 
 5   Product Subtype 1  550514 non-null  object 
 6   Product Subtype 2  403007 non-null  object 
 7   Product Subtype 3  437852 non-null  object 
 8   Product Subtype 4  68218 non-null   object 
 9   Manufacturer       551121 non-null  object 
 10  Family             551121 non-null  object 
 11  Model              551121 non-null  object 
 12  SKU                551121 non-null  object 
 13  Units              551121 non-null  float64
 14  Revenue            551121 non-null  float64
dtypes: float64(2), int64(3), object(10)
memory usage: 6

In [16]:
#Create a reordered mapped data file
result_map = pd.merge(pt_quarter, map_trim, how="left", on=["deviceID"])
result_map = result_map[result_map["Taxonomy"].notnull()]
result_map = result_map.iloc[:,[16,15,0,12,9,10,11,3,4,5,6,7,8,1,2,13,14]]
print(result_map.info())
print(result_map.head())

<class 'pandas.core.frame.DataFrame'>
Index: 361917 entries, 57 to 579033
Data columns (total 17 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Company            361917 non-null  object 
 1   Taxonomy           361917 non-null  object 
 2   deviceID           361917 non-null  int64  
 3   SKU                361917 non-null  object 
 4   Manufacturer       361917 non-null  object 
 5   Family             361917 non-null  object 
 6   Model              361917 non-null  object 
 7   MRG Market         361917 non-null  object 
 8   Product Type       361917 non-null  object 
 9   Product Subtype 1  361669 non-null  object 
 10  Product Subtype 2  292254 non-null  object 
 11  Product Subtype 3  327774 non-null  object 
 12  Product Subtype 4  61028 non-null   object 
 13  Year               361917 non-null  int64  
 14  Quarter            361917 non-null  int64  
 15  Units              361917 non-null  float64
 16  Revenu

In [17]:
#Export data to excel files
import openpyxl
import xlsxwriter
#Export mapped and unmapped SKUs for further processing
#with pd.ExcelWriter(out_path + "mapping.xlsx") as writer:
    pt_unmapped.to_excel(writer, sheet_name = "Unmapped SKUs", index=False)
    pt_mapped.to_excel(writer, sheet_name = "Mapped SKUs", index=False)
    taxonomy.to_excel(writer, sheet_name = "Taxonomy Key", index=False)
#Export mapped data to be used as a source for the PT Model tab in case no additional mapping is needed
#with pd.ExcelWriter(out_path + "mapped_data.xlsx") as writer:
    result_map.to_excel(writer, sheet_name = "Mapped Data", index=False)
#Export processed data to be used as a source for the PT Model tab
with pd.ExcelWriter(out_path + "PT_data.xlsx") as writer:
    pt_quarter.to_excel(writer, sheet_name = "PT Data", index=False)

In [19]:
#Create Pivot Table by Manufacturer
manf_summ = pd.pivot_table(pt_annual, values=["Revenue", "Units"], index=["Manufacturer"], columns=["Year"], aggfunc="sum")

#Sort by Revenue
manf_summ_rev_sort = manf_summ.reindex(manf_summ["Revenue"].sort_values(by=curr_year, ascending = False).index).map('{:,.0f}'.format)
print("Top 10 " + module + " Manufacturers Sorted by Revenue for Year: " + str(curr_year))

#Limit to top 10, add total revenue for each year at the bottom - for all manufacturers and print
# Top 10 rows from the sorted DataFrame
top_10_manf = manf_summ_rev_sort.head(10)

# Create a Grand Total row
grand_total_manf = manf_summ.sum(numeric_only=True).rename('Grand Total').map('{:,.0f}'.format)


# Convert the Series to a DataFrame for concatenation
grand_total_Manf_df = pd.DataFrame([grand_total_manf])

# Concatenate the top 10 rows with the Grand Total row
top10_manf_result = pd.concat([top_10_manf, grand_total_Manf_df])



# Display the result
display(top10_manf_result)

print()
#Create Pivot Table by Product Type
ptype_summ = pd.pivot_table(pt_annual, values=["Revenue", "Units"], index=["Product Type"], columns=["Year"], aggfunc="sum")
#Sort by Revenue
ptype_summ_rev_sort = ptype_summ.reindex(ptype_summ["Revenue"].sort_values(by=curr_year, ascending = False).index).map('{:,.0f}'.format)
print(module + " Product Segments Sorted by Revenue for Year: " + str(curr_year))
#Limit to top 10, add total revenue for each year at the bottom - for all manufacturers and print
# Top 10 rows from the sorted DataFrame
top_10_ptype = ptype_summ_rev_sort.head(10)

# Create a Grand Total row
grand_total_ptype = ptype_summ.sum(numeric_only=True).rename('Grand Total').map('{:,.0f}'.format)


# Convert the Series to a DataFrame for concatenation
grand_total_ptype_df = pd.DataFrame([grand_total_ptype])

# Concatenate the top 10 rows with the Grand Total row
top10_ptype_result = pd.concat([top_10_ptype, grand_total_ptype_df])
display(top10_ptype_result)






Top 10 Large Joint Recon Manufacturers Sorted by Revenue for Year: 2024


Revenue                                               \
Year                        2018           2019           2020           2021   
Zimmer Biomet      2,109,030,031  2,549,817,627  2,095,752,051  2,161,032,223   
Stryker            1,676,444,642  2,063,183,199  1,635,411,012  1,689,571,579   
Depuy Synthes      1,418,345,885  1,644,477,433  1,341,595,463  1,357,485,610   
Smith And Nephew     702,872,182    911,779,615    761,182,723    791,364,145   
Medacta               50,126,791     76,727,744     68,767,757     90,357,792   
Ortho Development     58,221,875     67,188,193     57,764,243     62,075,348   
Microport             83,721,422     81,173,511     62,722,270     70,286,806   
Exactech              45,290,947     59,885,395     60,416,437     60,081,301   
Djo Global            75,583,765    111,243,115    111,041,052    108,113,460   
Corin Group           68,726,808     69,390,938     32,415,663     30,776,133   
Grand Total        6,472,668,785  7,864,664,354  6,404,217,361  6,607,539,451   

                                                                               \
Year                        2022           2023           2024           2025   
Zimmer Biomet      2,364,369,953  2,535,570,276  2,372,696,333  1,418,022,019   
Stryker            1,809,664,512  1,992,304,730  1,971,647,988  1,125,463,556   
Depuy Synthes      1,405,538,503  1,464,483,203  1,473,177,751    828,720,184   
Smith And Nephew     763,550,911    786,744,033    786,081,531    402,597,416   
Medacta              102,625,685    112,728,736    109,886,555     65,947,693   
Ortho Development     59,324,657     56,357,956     69,741,731     40,881,720   
Microport             72,864,865     63,307,077     59,344,758     31,252,624   
Exactech              47,632,669     45,212,564     39,498,023     14,310,990   
Djo Global            19,789,182     25,550,066     28,863,849     13,483,771   
Corin Group           42,476,814     39,793,048     24,395,458     12,271,576   
Grand Total        6,871,060,943  7,323,243,491  7,130,393,350  4,065,458,000   

                       Units                                              \
Year                    2018       2019       2020       2021       2022   
Zimmer Biomet      1,686,271  2,121,643  1,830,112  1,955,980  2,201,521   
Stryker            1,294,772  1,655,893  1,341,175  1,435,889  1,559,641   
Depuy Synthes      1,133,259  1,385,030  1,209,576  1,274,780  1,355,574   
Smith And Nephew     498,899    647,462    572,971    596,243    598,625   
Medacta               37,157     63,497     69,747     92,246    111,197   
Ortho Development     55,981     59,685     52,513     57,837     61,947   
Microport             76,104     69,920     56,979     63,504     65,252   
Exactech              31,224     46,164     53,320     52,633     39,829   
Djo Global            63,679     96,772     99,811    104,186     16,977   
Corin Group           39,581     35,750     21,697     22,646     37,710   
Grand Total        6,586,089  8,618,531  7,813,946  9,153,510  9,368,271   

                                                      
Year                     2023        2024       2025  
Zimmer Biomet       2,464,679   2,322,675  1,481,111  
Stryker             1,765,167   1,722,504    969,827  
Depuy Synthes       1,423,704   1,439,189    839,695  
Smith And Nephew      613,909     601,735    334,055  
Medacta               115,137     112,904     69,046  
Ortho Development      58,559      71,092     43,567  
Microport              58,227      55,968     29,348  
Exactech               40,904      32,901     14,437  
Djo Global             20,145      19,321      9,997  
Corin Group            37,151      27,129     14,057  
Grand Total        10,814,419  10,739,544  7,070,566


Large Joint Recon Product Segments Sorted by Revenue for Year: 2024


Revenue                                               \
Year                  2018           2019           2020           2021   
Knees        3,876,656,502  4,682,991,867  3,700,777,264  3,830,153,038   
Hips         2,590,975,008  3,173,052,429  2,694,376,928  2,766,421,342   
Knee             1,720,153      3,977,643      5,372,142      7,298,722   
Instruments      3,313,373      4,495,786      3,286,946      3,057,689   
Augment              3,748        146,630        404,081        608,659   
Grand Total  6,472,668,785  7,864,664,354  6,404,217,361  6,607,539,451   

                                                                         \
Year                  2022           2023           2024           2025   
Knees        4,026,404,257  4,520,754,366  4,241,931,207  2,403,224,573   
Hips         2,829,806,461  2,782,994,956  2,863,595,977  1,648,614,810   
Knee            12,041,202     16,334,507     21,566,569     11,469,939   
Instruments      2,314,593      2,665,270      2,860,343      1,962,046   
Augment            494,430        494,392        439,255        186,632   
Grand Total  6,871,060,943  7,323,243,491  7,130,393,350  4,065,458,000   

                 Units                                              \
Year              2018       2019       2020       2021       2022   
Knees        3,918,763  5,226,743  4,902,048  5,575,076  5,861,267   
Hips         2,656,593  3,376,126  2,896,025  3,560,697  3,486,502   
Knee             1,338      3,387      4,547      6,141      9,488   
Instruments      9,393     12,201     11,163     11,316     10,785   
Augment              2         74        164        279        228   
Grand Total  6,586,089  8,618,531  7,813,946  9,153,510  9,368,271   

                                                
Year               2023        2024       2025  
Knees         7,156,879   7,148,295  4,996,114  
Hips          3,632,344   3,561,554  2,056,440  
Knee             13,163      15,742      8,425  
Instruments      11,778      13,735      9,472  
Augment             255         219        115  
Grand Total  10,814,419  10,739,544  7,070,566